In [1]:
import mne
import os.path as op
from tools import files
import numpy as np
import matplotlib.pylab as plt
from scipy.stats import trim_mean, ttest_1samp, sem
%matplotlib qt

In [2]:
# functions
trim = lambda x: trim_mean(x, 0.1, axis=0)

In [3]:
main_path = op.join("/cubric", "scratch", "c1557187", "act_mis")
meg_path = op.join(main_path, "MEG")
fs_path = "/cubric/scratch/c1557187/MRI_337/FS_OUTPUT"
results_path = op.join(main_path, "RESULTS", "WHATEVER_SOURCE_SPACE")

In [4]:
participants = files.get_folders_files(meg_path, wp=False)[0]
participants.sort()

In [ ]:
for participant in participants:
    print(participant)
    pp_path = op.join(meg_path, participant, "new_v1")
    all_epochs = files.get_files(pp_path, "epochs-TD", "-epo.fif")[2]
    all_epochs.sort()
    fwd_solution = files.get_files(pp_path, "", "-fwd.fif")[2]
    fwd_solution.sort()
    noise_cov = files.get_files(pp_path, "epochs-TD", "-cov.fif")[2]
    noise_cov.sort()
    for ix in range(2):
        epochs = mne.read_epochs(all_epochs[ix])
        cov = mne.read_cov(noise_cov[ix])
        fwd = mne.read_forward_solution(fwd_solution[ix])
        
        reg_avg = epochs["30"].average(method=trim)
        odd_avg = epochs["40"].average(method=trim)
        diff_avg = mne.combine_evoked([odd_avg, -reg_avg], weights=[30/140, 110/140])
        
        inverse_operator = mne.minimum_norm.make_inverse_operator(
            epochs.info,
            fwd,
            cov,
            loose=0.2,
            depth=0.8
        )
        
        reg_stc = mne.minimum_norm.apply_inverse(
            reg_avg,
            inverse_operator,
            lambda2= 1/3**2,
            method="dSPM"
        )
        odd_stc = mne.minimum_norm.apply_inverse(
            reg_avg,
            inverse_operator,
            lambda2= 1/3**2,
            method="dSPM"
        )
        diff_stc = mne.minimum_norm.apply_inverse(
            reg_avg,
            inverse_operator,
            lambda2= 1/3**2,
            method="dSPM"
        )
        
        morph = mne.compute_source_morph(
            reg_stc, 
            subject_from=participant,
            subject_to="fsaverage",
            subjects_dir=fs_path
        )
        
        reg_fsa = morph.apply(reg_stc)
        odd_fsa = morph.apply(odd_stc)
        diff_fsa = morph.apply(diff_stc)
        ix_n = str(ix).zfill(3)
        reg_path = op.join(results_path, "reg-{}-{}".format(ix_n, participant))
        odd_path = op.join(results_path, "odd-{}-{}".format(ix_n, participant))
        diff_path = op.join(results_path, "dif-{}-{}".format(ix_n, participant))
        reg_stc.save(reg_path, ftype="stc")
        odd_stc.save(odd_path, ftype="stc")
        diff_stc.save(diff_path, ftype="stc")
        

0001
0003
0004
0008
0009
0014
0015
0016
0017
0019
0021
0027
0030
0032
0034
0035
0038
0039
0040
0041
0042
0043
